## UK Deaths by Vaccination Status

This notebook plots the May 2022 dataset from https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/deathsbyvaccinationstatusengland.
Curiously they stopped reporting the data, so let's take a look at it.

The .xlsx contains a bunch of tabs, but the interesting one contains standardized numbers per 100,000 which is what you need to do if you want to compare the numbers.  So we take "Table 3" which is "Whole period age-standardised mortality rates by vaccination status for all cause deaths, deaths involving COVID-19 and deaths not involving COVID-19, per 100,000 person-years, England, deaths occurring between 1 January 2021 and 31 May 2022".

Then this code pivots on vaccination status so we can pull out the data for Unvaccinated and compare that with all the other kinds of vaccination status they break out, which are:

- First dose, less than 21 days ago
- First dose, at least 21 days ago
- Second dose, less than 21 days ago
- Second dose, between 21 days and 6 months ago
- Second dose, at least 6 months ago
- Third dose or booster, less than 21 days ago
- Third dose or booster, at least 21 days ago
- Ever vaccinated

Now what I don't understand is that 'Ever vaccinated' should essentially be showing the average of all the other columns, but as you can see below it is not.  I computed the average myself (see the last chart) and it does not match.  The other problem with this kind of "averaging" is that one of the vaccinated groups named "Second dose, at least 6 months ago" cannot start until halfway through 2021 by definition, so it starts with all zeros, and those zeros are artificially pulling down the average for the vaccinated group.  So what we really have to do is look at all the charts and they all look pretty bad, except "Third dose or booster, at least 21 days ago" but even that is converging with unvaccinated by May 2022 when the data stops, and I suspect it crossed that path and is now much worse also which is probably why they stopped tracking it.

When you look at all the charts, it also shows many of these vaccinated groups are doing very poorly.


In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

data = pd.read_csv('table.csv')
df = pd.DataFrame(data, columns=['Year', 'Month', 'Vaccination status', 'Age-standardised mortality rate / 100,000 person-years'])

# pivot table to break status out as columns
columns = ['Date']
for i, row in df.iterrows():
    status = row['Vaccination status']
    if not status in columns:
        columns += [status]   

dp = pd.DataFrame(columns=columns)

data = {}
for i, row in df.iterrows():
    year = row['Year']
    month = row['Month']
    status = row['Vaccination status']
    rate = row['Age-standardised mortality rate / 100,000 person-years']
    if rate == 'x':
        rate = 0
    rate = float(rate)
    
    if status == 'Unvaccinated':
        data['Date'] = f"{year}.{month}"

    data[status] = rate

    if status == 'Ever vaccinated':
        dp = dp.append(data, ignore_index=True)
        data = {}

dates = dp[columns[0]]
xs = list(range(len(dates)))

averages = np.zeros(len(dates))
count = 0
for i in range(2, len(columns)):
    unvax = dp[columns[1]]
    label = columns[i]
    vax = dp[label]
    vs = np.array(vax)
    if label != 'Ever vaccinated':
        averages += vs
        count += 1
    p = figure(width=1000, height=400) 
    p.line(xs, unvax, color='blue', legend_label=columns[1])
    p.line(xs, vax, color='green', legend_label=label)
    show(p)

averages /= count # average back to per 100k
p = figure(width=1000, height=400) 
p.line(xs, unvax, color='blue', legend_label=columns[1])
p.line(xs, averages, color='green', legend_label="Average vaccinated")
show(p)


Loading BokehJS ...